# 第4章 变形

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


## 一、透视表
### 1. pivot
#### 一般状态下，数据在DataFrame会以压缩（stacked）状态存放，例如上面的Gender，两个类别被叠在一列中，pivot函数可将某一列作为新的cols：

In [2]:
# pivot就是将有多个唯一值的列, 按照唯一值的数量分为相应数量的列, 并将相应原始列的取值对应的新列的值设置为非 NaN
# 此外提供了其他两个参数: index参数可以指定索引, values参数可以使用别的值对展开的列进行填充
df.pivot(index='ID',columns='Gender',values='Height').head()

Gender,F,M
ID,,
1101,NaN,173.0
1102,192.0,NaN
1103,NaN,186.0
1104,167.0,NaN
1105,159.0,NaN


In [3]:
df.pivot??

Signature: df.pivot(index=None, columns=None, values=None) -> 'DataFrame'
Docstring:
Return reshaped DataFrame organized by given index / column values.

Reshape data (produce a "pivot" table) based on column values. Uses
unique values from specified `index` / `columns` to form axes of the
resulting DataFrame. This function does not support data
aggregation, multiple values will result in a MultiIndex in the
columns. See the :ref:`User Guide <reshaping>` for more on reshaping.

Parameters
----------
index : str or object, optional
    Column to use to make new frame's index. If None, uses
    existing index.
columns : str or object
    Column to use to make new frame's columns.
values : str, object or a list of the previous, optional
    Column(s) to use for populating new frame's values. If not
    specified, all remaining columns will be used and the result will
    have hierarchically indexed columns.

    .. versionchanged:: 0.23.0
       Also accept list of column names.

Returns


In [9]:
# 一个稍微正常且有用的示例
df.pivot(index='ID',columns='Gender',values='Gender').head()

Gender,F,M
ID,,
1101,NaN,M
1102,F,NaN
1103,NaN,M
1104,F,NaN
1105,F,NaN


In [45]:
# 唯一必备的参数是 columns
df.pivot(columns='Gender').head()

School      Class           ID           Address           Height  \
Gender      F    M     F    M       F       M         F         M      F   
0         NaN  S_1   NaN  C_1     NaN  1101.0       NaN  street_1    NaN   
1         S_1  NaN   C_1  NaN  1102.0     NaN  street_2       NaN  192.0   
2         NaN  S_1   NaN  C_1     NaN  1103.0       NaN  street_2    NaN   
3         S_1  NaN   C_1  NaN  1104.0     NaN  street_2       NaN  167.0   
4         S_1  NaN   C_1  NaN  1105.0     NaN  street_4       NaN  159.0   

              Weight        Math       Physics       
Gender      M      F     M     F     M       F    M  
0       173.0    NaN  63.0   NaN  34.0     NaN   A+  
1         NaN   73.0   NaN  32.5   NaN      B+  NaN  
2       186.0    NaN  82.0   NaN  87.2     NaN   B+  
3         NaN   81.0   NaN  80.4   NaN      B-  NaN  
4         NaN   64.0   NaN  84.8   NaN      B+  NaN

In [46]:
# 如果想知道不同性别的学生的某一特征之间是否有差异,例如平均体重, 则可接着直接对体重下的两个列求均值
df.pivot(columns='Gender').mean()

        Gender
ID      F         1824.368421
        M         1777.625000
Height  F          173.473684
        M          174.937500
Weight  F           73.315789
        M           76.250000
Math    F           65.447368
        M           56.487500
dtype: float64

In [50]:
# 如果想知道不同学校的不同性别的学生的平均体重等差异, 则可分组后求均值
df.pivot(columns='Gender').groupby(df.School).mean()
# 如果只想知道其中某一项或某几项的均值, 则可以使用列的层次索引定位相应的列--或者直接在pivot时使用相应的列值进行填充(这时列名会是转置列的唯一值)

ID                   Height                 Weight  \
Gender            F            M           F           M          F   
School                                                                
S_1     1191.125000  1216.571429  173.125000  178.714286  70.500000   
S_2     2284.909091  2214.000000  173.727273  172.000000  75.363636   

                        Math             
Gender          M          F          M  
School                                   
S_1     69.428571  64.100000  63.342857  
S_2     81.555556  66.427273  51.155556

In [51]:
# 对不同学校的学生, 按性别统计平均数学成绩
# 首先使用pivot按照Gender列转置, 将填充值设置为Math, 然后按照df.Scholl分组求均值.
df.pivot(columns='Gender',values='Math').groupby(df.School).mean()

Gender,F,M
School,,
S_1,64.100000,63.342857
S_2,66.427273,51.155556


#### 然而pivot函数具有很强的局限性，除了功能上较少之外，还不允许values中出现重复的行列索引对（pair），例如下面的语句就会报错：

In [52]:
df.pivot(index='School',columns='Gender',values='Height').head()
# ValueError: Index contains duplicate entries, cannot reshape
# 不能设置重复索引

ValueError: Index contains duplicate entries, cannot reshape

In [53]:
df.pivot(columns='Gender',values='Height').head()
# 不传入索引, 就会继续用默认整数索引

Gender,F,M
0,NaN,173.0
1,192.0,NaN
2,NaN,186.0
3,167.0,NaN
4,159.0,NaN


In [54]:
# 也可以在转置后, 把有用的列设置为层次索引, 接下来可以根据需要按照索引分组聚合
df_mul=df.pivot(columns='Gender',values='Height')
df_mul.head()
df_mul.index=pd.MultiIndex.from_tuples(zip(list(df['School'].values),list(df['Class'].values)),names=('School','Class'))
df_mul

Gender            F      M
School Class              
S_1    C_1      NaN  173.0
       C_1    192.0    NaN
       C_1      NaN  186.0
       C_1    167.0    NaN
       C_1    159.0    NaN
       C_2      NaN  188.0
       C_2    176.0    NaN
       C_2      NaN  160.0
       C_2    162.0    NaN
       C_2    167.0    NaN
       C_3      NaN  161.0
       C_3    175.0    NaN
       C_3      NaN  188.0
       C_3      NaN  195.0
       C_3    187.0    NaN
S_2    C_1      NaN  174.0
       C_1    161.0    NaN
       C_1      NaN  157.0
       C_1    159.0    NaN
       C_1      NaN  170.0
       C_2      NaN  193.0
       C_2    194.0    NaN
       C_2      NaN  155.0
       C_2      NaN  175.0
       C_2    183.0    NaN
       C_3    157.0    NaN
       C_3      NaN  171.0
       C_3    190.0    NaN
       C_3    164.0    NaN
       C_3      NaN  187.0
       C_4    192.0    NaN
       C_4      NaN  166.0
       C_4    158.0    NaN
       C_4    160.0    NaN
       C_4    193.0    NaN

In [55]:
# 另一个思路是, 将pivot的结果与原始df进行join
# 例如, 若想知道不同学校不同性别的学生的平均体重
df.pivot(columns='Gender',values='Height').join(df).head()

,F,M,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,NaN,173.0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,192.0,NaN,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,NaN,186.0,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,167.0,NaN,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,159.0,NaN,S_1,C_1,1105,F,street_4,159,64,84.8,B+


### 2. pivot_table
#### 因此，更多的时候会选择使用强大的pivot_table函数
#### 首先，再现上面的操作：

In [56]:
pd.pivot_table??

Signature:
pd.pivot_table(
    data,
    values=None,
    index=None,
    columns=None,
    aggfunc='mean',
    fill_value=None,
    margins=False,
    dropna=True,
    margins_name='All',
    observed=False,
) -> 'DataFrame'
Docstring:
Create a spreadsheet-style pivot table as a DataFrame.

The levels in the pivot table will be stored in MultiIndex objects
(hierarchical indexes) on the index and columns of the result DataFrame.

Parameters
----------
data : DataFrame
values : column to aggregate, optional
index : column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The
    list can contain any of the other types (except list).
    Keys to group by on the pivot table index.  If an array is passed,
    it is being used as the same manner as column values.
columns : column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The
    list can contain any of the other types 

In [57]:
pd.pivot_table(df,index='ID',columns='Gender',values='Height').head()

Gender,F,M
ID,,
1101,NaN,173.0
1102,192.0,NaN
1103,NaN,186.0
1104,167.0,NaN
1105,159.0,NaN


#### 由于功能更多，速度上自然是比不上原来的pivot函数：

In [58]:
%timeit df.pivot(index='ID',columns='Gender',values='Height')
%timeit pd.pivot_table(df,index='ID',columns='Gender',values='Height')

1.56 ms ± 44.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.63 ms ± 127 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Pandas中提供了各种选项，下面介绍常用参数：

#### ① aggfunc：对组内进行聚合统计，可传入各类函数，默认为'mean'

In [59]:
# index可以设置为有重复值的列--pivot是不可以的
# 比pivot多的 aggfunc 参数, 如果被传入了相应的值, 会自动按照索引进行分组,然后在组内应用传入的聚合函数
# 按学校求Height的:均值,总和
pd.pivot_table(df,index='School',columns='Gender',values='Height',aggfunc=['mean','sum']).head()

mean               sum      
Gender           F           M     F     M
School                                    
S_1     173.125000  178.714286  1385  1251
S_2     173.727273  172.000000  1911  1548

In [62]:
# 将索引设置为班级, 则会按照班级进行分组--注意这种分组方法将不同学校的同一个班级名称的放到一起了,这不合理
pd.pivot_table(df,index='Class',columns='Gender',values='Height',aggfunc=['mean','sum'])

mean         sum     
Gender       F      M    F    M
Class                          
C_1     167.60  172.0  838  860
C_2     176.40  174.2  882  871
C_3     174.60  180.4  873  902
C_4     175.75  166.0  703  166

In [63]:
# 也可以传入层次索引,则会按照层次索引分组聚合
pd.pivot_table(df,index=['School','Class'],columns='Gender',values='Height',aggfunc=['mean','sum'])

mean              sum     
Gender                 F           M    F    M
School Class                                  
S_1    C_1    172.666667  179.500000  518  359
       C_2    168.333333  174.000000  505  348
       C_3    181.000000  181.333333  362  544
S_2    C_1    160.000000  167.000000  320  501
       C_2    188.500000  174.333333  377  523
       C_3    170.333333  179.000000  511  358
       C_4    175.750000  166.000000  703  166

#### ② margins：汇总边际状态

In [64]:
# margins参数为True的时候, 除了会按照分组键应用聚合函数, 还会增加一个小计列
# 注意行和列都增加了小计行 all
pd.pivot_table(df,index='School',columns='Gender',values='Height',aggfunc=['mean','sum'],margins=True).head()
#margins_name可以设置名字，默认为'All'

mean                           sum            
Gender           F           M         All     F     M   All
School                                                      
S_1     173.125000  178.714286  175.733333  1385  1251  2636
S_2     173.727273  172.000000  172.950000  1911  1548  3459
All     173.473684  174.937500  174.142857  3296  2799  6095

In [65]:
pd.pivot_table(df,index=['School','Class'],columns='Gender',values='Height',aggfunc=['mean','sum'], margins=True)
# 但是层次化索引的情况下, 并没有按第一层级的索引进行聚合, 而是把所有行聚合到了一个all里了---注意这和列的情况是不一样的

mean                           sum            
Gender                 F           M         All     F     M   All
School Class                                                      
S_1    C_1    172.666667  179.500000  175.400000   518   359   877
       C_2    168.333333  174.000000  170.600000   505   348   853
       C_3    181.000000  181.333333  181.200000   362   544   906
S_2    C_1    160.000000  167.000000  164.200000   320   501   821
       C_2    188.500000  174.333333  180.000000   377   523   900
       C_3    170.333333  179.000000  173.800000   511   358   869
       C_4    175.750000  166.000000  173.800000   703   166   869
All           173.473684  174.937500  174.142857  3296  2799  6095

#### ③ 行、列、值都可以为多级

In [66]:
# 分别为 index, columns和values传入原始df的列名构成的list, 则可以指定层次化行列索引,以及多个列的值进行填充
pd.pivot_table(df,index=['School','Class'],
               columns=['Gender','Address'],
               values=['Height','Weight'])
# 具体的案例, 还需要根据需求进行合理设计

Height                                                        \
Gender              F                                                     M   
Address      street_1 street_2 street_4 street_5 street_6 street_7 street_1   
School Class                                                                  
S_1    C_1        NaN    179.5    159.0      NaN      NaN      NaN    173.0   
       C_2        NaN      NaN    176.0    162.0    167.0      NaN      NaN   
       C_3      175.0      NaN      NaN    187.0      NaN      NaN      NaN   
S_2    C_1        NaN      NaN      NaN    159.0    161.0      NaN      NaN   
       C_2        NaN      NaN      NaN      NaN      NaN    188.5    175.0   
       C_3        NaN      NaN    157.0      NaN    164.0    190.0      NaN   
       C_4        NaN    176.0      NaN      NaN    175.5      NaN      NaN   

                                         ...   Weight                    \
Gender                                   ...        F                     
Address      street_2 street_4 street_5  ... street_4 street_5 street_6   
School Class                             ...                              
S_1    C_1      186.0      NaN      NaN  ...     64.0      NaN      NaN   
       C_2        NaN      NaN    188.0  ...     94.0     63.0     63.0   
       C_3      195.0    161.0      NaN  ...      NaN     69.0      NaN   
S_2    C_1        NaN    163.5      NaN  ...      NaN     97.0     61.0   
       C_2        NaN    155.0    193.0  ...      NaN      NaN      NaN   
       C_3        NaN    187.0    171.0  ...     78.0      NaN     81.0   
       C_4        NaN      NaN      NaN  ...      NaN      NaN     57.0   

                                                                             
Gender                       M                                               
Address      street_7 street_1 street_2 street_4 street_5 street_6 street_7  
School Class                                                                 
S_1    C_1        NaN     63.0     82.0      NaN      NaN      NaN      NaN  
       C_2        NaN      NaN      NaN      NaN     68.0     53.0      NaN  
       C_3        NaN      NaN     70.0     68.0      NaN      NaN     82.0  
S_2    C_1        NaN      NaN      NaN     71.0      NaN      NaN     84.0  
       C_2       76.5     74.0      NaN     91.0    100.0      NaN      NaN  
       C_3       99.0      NaN      NaN     73.0     88.0      NaN      NaN  
       C_4        NaN      NaN      NaN      NaN      NaN      NaN     82.0  

[7 rows x 24 columns]

In [73]:
pd.pivot_table(df,index=['School','Address'],
               columns=['Gender','Class'],
               values=['Height','Weight'])
# [(S_1,street_2),(Height,F,C_1)] 位置的元素是179.5

Height                                                   \
Gender               F                           M                        
Class              C_1    C_2    C_3    C_4    C_1    C_2    C_3    C_4   
School Address                                                            
S_1    street_1    NaN    NaN  175.0    NaN  173.0    NaN    NaN    NaN   
       street_2  179.5    NaN    NaN    NaN  186.0    NaN  195.0    NaN   
       street_4  159.0  176.0    NaN    NaN    NaN    NaN  161.0    NaN   
       street_5    NaN  162.0  187.0    NaN    NaN  188.0    NaN    NaN   
       street_6    NaN  167.0    NaN    NaN    NaN  160.0    NaN    NaN   
       street_7    NaN    NaN    NaN    NaN    NaN    NaN  188.0    NaN   
S_2    street_1    NaN    NaN    NaN    NaN    NaN  175.0    NaN    NaN   
       street_2    NaN    NaN    NaN  176.0    NaN    NaN    NaN    NaN   
       street_4    NaN    NaN  157.0    NaN  163.5  155.0  187.0    NaN   
       street_5  159.0    NaN    NaN    NaN    NaN  193.0  171.0    NaN   
       street_6  161.0    NaN  164.0  175.5    NaN    NaN    NaN    NaN   
       street_7    NaN  188.5  190.0    NaN  174.0    NaN    NaN  166.0   

                Weight                                             
Gender               F                       M                     
Class              C_1   C_2   C_3   C_4   C_1    C_2   C_3   C_4  
School Address                                                     
S_1    street_1    NaN   NaN  57.0   NaN  63.0    NaN   NaN   NaN  
       street_2   77.0   NaN   NaN   NaN  82.0    NaN  70.0   NaN  
       street_4   64.0  94.0   NaN   NaN   NaN    NaN  68.0   NaN  
       street_5    NaN  63.0  69.0   NaN   NaN   68.0   NaN   NaN  
       street_6    NaN  63.0   NaN   NaN   NaN   53.0   NaN   NaN  
       street_7    NaN   NaN   NaN   NaN   NaN    NaN  82.0   NaN  
S_2    street_1    NaN   NaN   NaN   NaN   NaN   74.0   NaN   NaN  
       street_2    NaN   NaN   NaN  73.0   NaN    NaN   NaN   NaN  
       street_4    NaN   NaN  78.0   NaN  71.0   91.0  73.0   NaN  
       street_5   97.0   NaN   NaN   NaN   NaN  100.0  88.0   NaN  
       street_6   61.0   NaN  81.0  57.0   NaN    NaN   NaN   NaN  
       street_7    NaN  76.5  99.0   NaN  84.0    NaN   NaN  82.0

In [77]:
# 注意传入聚合函数和不传入聚合函数, 相同位置的单元格的差别--[(S_1,street_2),(Height,F,C_1)] 位置的元素是359.0
pd.pivot_table(df,index=['School','Address'],
               columns=['Gender','Class'],
               values=['Height','Weight'],aggfunc=['sum'])

sum                                                   \
                Height                                                    
Gender               F                           M                        
Class              C_1    C_2    C_3    C_4    C_1    C_2    C_3    C_4   
School Address                                                            
S_1    street_1    NaN    NaN  175.0    NaN  173.0    NaN    NaN    NaN   
       street_2  359.0    NaN    NaN    NaN  186.0    NaN  195.0    NaN   
       street_4  159.0  176.0    NaN    NaN    NaN    NaN  161.0    NaN   
       street_5    NaN  162.0  187.0    NaN    NaN  188.0    NaN    NaN   
       street_6    NaN  167.0    NaN    NaN    NaN  160.0    NaN    NaN   
       street_7    NaN    NaN    NaN    NaN    NaN    NaN  188.0    NaN   
S_2    street_1    NaN    NaN    NaN    NaN    NaN  175.0    NaN    NaN   
       street_2    NaN    NaN    NaN  352.0    NaN    NaN    NaN    NaN   
       street_4    NaN    NaN  157.0    NaN  327.0  155.0  187.0    NaN   
       street_5  159.0    NaN    NaN    NaN    NaN  193.0  171.0    NaN   
       street_6  161.0    NaN  164.0  351.0    NaN    NaN    NaN    NaN   
       street_7    NaN  377.0  190.0    NaN  174.0    NaN    NaN  166.0   

                                                                      
                Weight                                                
Gender               F                          M                     
Class              C_1    C_2   C_3    C_4    C_1    C_2   C_3   C_4  
School Address                                                        
S_1    street_1    NaN    NaN  57.0    NaN   63.0    NaN   NaN   NaN  
       street_2  154.0    NaN   NaN    NaN   82.0    NaN  70.0   NaN  
       street_4   64.0   94.0   NaN    NaN    NaN    NaN  68.0   NaN  
       street_5    NaN   63.0  69.0    NaN    NaN   68.0   NaN   NaN  
       street_6    NaN   63.0   NaN    NaN    NaN   53.0   NaN   NaN  
       street_7    NaN    NaN   NaN    NaN    NaN    NaN  82.0   NaN  
S_2    street_1    NaN    NaN   NaN    NaN    NaN   74.0   NaN   NaN  
       street_2    NaN    NaN   NaN  146.0    NaN    NaN   NaN   NaN  
       street_4    NaN    NaN  78.0    NaN  142.0   91.0  73.0   NaN  
       street_5   97.0    NaN   NaN    NaN    NaN  100.0  88.0   NaN  
       street_6   61.0    NaN  81.0  114.0    NaN    NaN   NaN   NaN  
       street_7    NaN  153.0  99.0    NaN   84.0    NaN   NaN  82.0

In [79]:
# 注意传入聚合函数和不传入聚合函数, 相同位置的单元格的差别--[(S_1,street_2),(Height,F,C_1)] 位置的元素是2.0
pd.pivot_table(df,index=['School','Address'],
               columns=['Gender','Class'],
               values=['Height','Weight'],aggfunc=['count'])

count                                                      \
                Height                                    Weight             
Gender               F                   M                     F             
Class              C_1  C_2  C_3  C_4  C_1  C_2  C_3  C_4    C_1  C_2  C_3   
School Address                                                               
S_1    street_1    NaN  NaN  1.0  NaN  1.0  NaN  NaN  NaN    NaN  NaN  1.0   
       street_2    2.0  NaN  NaN  NaN  1.0  NaN  1.0  NaN    2.0  NaN  NaN   
       street_4    1.0  1.0  NaN  NaN  NaN  NaN  1.0  NaN    1.0  1.0  NaN   
       street_5    NaN  1.0  1.0  NaN  NaN  1.0  NaN  NaN    NaN  1.0  1.0   
       street_6    NaN  1.0  NaN  NaN  NaN  1.0  NaN  NaN    NaN  1.0  NaN   
       street_7    NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN    NaN  NaN  NaN   
S_2    street_1    NaN  NaN  NaN  NaN  NaN  1.0  NaN  NaN    NaN  NaN  NaN   
       street_2    NaN  NaN  NaN  2.0  NaN  NaN  NaN  NaN    NaN  NaN  NaN   
       street_4    NaN  NaN  1.0  NaN  2.0  1.0  1.0  NaN    NaN  NaN  1.0   
       street_5    1.0  NaN  NaN  NaN  NaN  1.0  1.0  NaN    1.0  NaN  NaN   
       street_6    1.0  NaN  1.0  2.0  NaN  NaN  NaN  NaN    1.0  NaN  1.0   
       street_7    NaN  2.0  1.0  NaN  1.0  NaN  NaN  1.0    NaN  2.0  1.0   

                                          
                                          
Gender                  M                 
Class            C_4  C_1  C_2  C_3  C_4  
School Address                            
S_1    street_1  NaN  1.0  NaN  NaN  NaN  
       street_2  NaN  1.0  NaN  1.0  NaN  
       street_4  NaN  NaN  NaN  1.0  NaN  
       street_5  NaN  NaN  1.0  NaN  NaN  
       street_6  NaN  NaN  1.0  NaN  NaN  
       street_7  NaN  NaN  NaN  1.0  NaN  
S_2    street_1  NaN  NaN  1.0  NaN  NaN  
       street_2  2.0  NaN  NaN  NaN  NaN  
       street_4  NaN  2.0  1.0  1.0  NaN  
       street_5  NaN  NaN  1.0  1.0  NaN  
       street_6  2.0  NaN  NaN  NaN  NaN  
       street_7  NaN  1.0  NaN  NaN  1.0

### 3. crosstab（交叉表）
#### 交叉表是一种特殊的透视表，典型的用途如分组统计，如现在想要统计关于街道和性别分组的频数：

In [82]:
pd.crosstab(index=df['Address'],columns=df['Gender']) # 未传入函数, 默认是count

Gender,F,M
Address,,
street_1,1,2
street_2,4,2
street_4,3,5
street_5,3,3
street_6,5,1
street_7,3,3


In [90]:
pd.crosstab(index=df['Address'],columns=df['Gender'], aggfunc='mean', values=df['Math']) # 传入非计数的聚合函数, 则必须也传入values

Gender,F,M
Address,,
street_1,87.700000,40.600000
street_2,56.475000,86.200000
street_4,73.533333,48.180000
street_5,55.900000,56.266667
street_6,64.360000,58.800000
street_7,73.266667,60.566667


In [81]:
pd.crosstab??

Signature:
pd.crosstab(
    index,
    columns,
    values=None,
    rownames=None,
    colnames=None,
    aggfunc=None,
    margins=False,
    margins_name: str = 'All',
    dropna: bool = True,
    normalize=False,
) -> 'DataFrame'
Source:   
def crosstab(
    index,
    columns,
    values=None,
    rownames=None,
    colnames=None,
    aggfunc=None,
    margins=False,
    margins_name: str = "All",
    dropna: bool = True,
    normalize=False,
) -> "DataFrame":
    """
    Compute a simple cross tabulation of two (or more) factors. By default
    computes a frequency table of the factors unless an array of values and an
    aggregation function are passed.

    Parameters
    ----------
    index : array-like, Series, or list of arrays/Series
        Values to group by in the rows.
    columns : array-like, Series, or list of arrays/Series
        Values to group by in the columns.
    values : array-like, optional
        Array of values to aggregate according to the factors.
    

In [95]:
# 为index和columns传入原始df的列构成的list, 则可以按照该list构造层次索引
pd.crosstab(index=[df.School,df.Address],columns=[df.Gender,df.Class])

Gender            F               M            
Class           C_1 C_2 C_3 C_4 C_1 C_2 C_3 C_4
School Address                                 
S_1    street_1   0   0   1   0   1   0   0   0
       street_2   2   0   0   0   1   0   1   0
       street_4   1   1   0   0   0   0   1   0
       street_5   0   1   1   0   0   1   0   0
       street_6   0   1   0   0   0   1   0   0
       street_7   0   0   0   0   0   0   1   0
S_2    street_1   0   0   0   0   0   1   0   0
       street_2   0   0   0   2   0   0   0   0
       street_4   0   0   1   0   2   1   1   0
       street_5   1   0   0   0   0   1   1   0
       street_6   1   0   1   2   0   0   0   0
       street_7   0   2   1   0   1   0   0   1

In [99]:
# 传入非计数的聚合函数, 则必须也传入values
pd.crosstab(index=[df.School,df.Class],columns=[df.Gender,df.Address], aggfunc='mean', values=df['Math'])#.fillna(0) # 缺失值的填充则要充分考虑业务需求

Gender              F                                                     M  \
Address      street_1 street_2 street_4 street_5 street_6 street_7 street_1   
School Class                                                                  
S_1    C_1        NaN    56.45     84.8      NaN      NaN      NaN     34.0   
       C_2        NaN      NaN     63.5     33.8    68.40      NaN      NaN   
       C_3       87.7      NaN      NaN     61.7      NaN      NaN      NaN   
S_2    C_1        NaN      NaN      NaN     72.2    50.60      NaN      NaN   
       C_2        NaN      NaN      NaN      NaN      NaN    76.95     47.2   
       C_3        NaN      NaN     72.3      NaN    95.50    65.90      NaN   
       C_4        NaN    56.50      NaN      NaN    53.65      NaN      NaN   

Gender                                                     
Address      street_2 street_4 street_5 street_6 street_7  
School Class                                               
S_1    C_1       87.2      NaN      NaN      NaN      NaN  
       C_2        NaN      NaN     97.0     58.8      NaN  
       C_3       85.2    31.50      NaN      NaN     49.7  
S_2    C_1        NaN    43.35      NaN      NaN     83.3  
       C_2        NaN    73.80     39.1      NaN      NaN  
       C_3        NaN    48.90     32.7      NaN      NaN  
       C_4        NaN      NaN      NaN      NaN     48.7

#### 交叉表的功能也很强大（但目前还不支持多级分组），下面说明一些重要参数：
#### ① values和aggfunc：分组对某些数据进行聚合操作，这两个参数必须成对出现

In [100]:
pd.crosstab(index=df['Address'],columns=df['Gender'],
            values=np.random.randint(1,20,df.shape[0]),aggfunc='min')
#默认参数等于如下方法：
#pd.crosstab(index=df['Address'],columns=df['Gender'],values=1,aggfunc='count')

Gender,F,M
Address,,
street_1,12,5
street_2,2,3
street_4,1,7
street_5,5,5
street_6,4,5
street_7,2,5


In [101]:
# 与上述方式等价--默认函数是count,因此传入任何(非缺失)值都可以
pd.crosstab(index=df['Address'],columns=df['Gender'],values=1,aggfunc='count')

Gender,F,M
Address,,
street_1,1,2
street_2,4,2
street_4,3,5
street_5,3,3
street_6,5,1
street_7,3,3


#### ② 除了边际参数margins外，还引入了normalize参数，可选'all','index','columns'参数值

In [102]:
pd.crosstab(index=df['Address'],columns=df['Gender'],normalize='all',margins=True)
# normalize : bool, {'all', 'index', 'columns'}, or {0,1}, default False
#   Normalize by dividing all values by the sum of values.
#        - If passed 'all' or `True`, will normalize over all values.
#        - If passed 'index' will normalize over each row.
#        - If passed 'columns' will normalize over each column.
#        - If margins is `True`, will also normalize margin values.

Gender,F,M,All
Address,,,
street_1,0.028571,0.057143,0.085714
street_2,0.114286,0.057143,0.171429
street_4,0.085714,0.142857,0.228571
street_5,0.085714,0.085714,0.171429
street_6,0.142857,0.028571,0.171429
street_7,0.085714,0.085714,0.171429
All,0.542857,0.457143,1.000000


In [103]:
pd.crosstab(index=df['Address'],columns=df['Gender'],normalize='all')


Gender,F,M
Address,,
street_1,0.028571,0.057143
street_2,0.114286,0.057143
street_4,0.085714,0.142857
street_5,0.085714,0.085714
street_6,0.142857,0.028571
street_7,0.085714,0.085714


## 二、其他变形方法
### 1. melt
#### melt函数可以认为是pivot函数的逆操作，将unstacked状态的数据，压缩成stacked，使“宽”的DataFrame变“窄”

In [118]:
df_m = df[['ID','Gender','Math']]
df_m.head()

,ID,Gender,Math
0,1101,M,34.0
1,1102,F,32.5
2,1103,M,87.2
3,1104,F,80.4
4,1105,F,84.8


In [119]:
df_p=df_m.pivot(index='ID',columns='Gender',values='Math')#.head()
df_p.head()

Gender,F,M
ID,,
1101,NaN,34.0
1102,32.5,NaN
1103,NaN,87.2
1104,80.4,NaN
1105,84.8,NaN


In [121]:
df_p.reset_index().head()

Gender,ID,F,M
0,1101,NaN,34.0
1,1102,32.5,NaN
2,1103,NaN,87.2
3,1104,80.4,NaN
4,1105,84.8,NaN


In [122]:
df_p.reset_index().melt(id_vars=['ID'],value_vars=['F','M'],value_name='Math')

,ID,Gender,Math
0,1101,F,NaN
1,1102,F,32.5
2,1103,F,NaN
3,1104,F,80.4
4,1105,F,84.8
...,...,...,...
65,2401,M,NaN
66,2402,M,48.7
67,2403,M,NaN
68,2404,M,NaN


In [123]:
df_p.reset_index().melt(id_vars=['ID'],value_vars=['F','M'],value_name='Math').dropna()

,ID,Gender,Math
1,1102,F,32.5
3,1104,F,80.4
4,1105,F,84.8
6,1202,F,63.5
8,1204,F,33.8
9,1205,F,68.4
11,1302,F,87.7
14,1305,F,61.7
16,2102,F,50.6
18,2104,F,72.2


In [124]:
df_m

,ID,Gender,Math
0,1101,M,34.0
1,1102,F,32.5
2,1103,M,87.2
3,1104,F,80.4
4,1105,F,84.8
5,1201,M,97.0
6,1202,F,63.5
7,1203,M,58.8
8,1204,F,33.8
9,1205,F,68.4


#### melt函数中的id_vars表示需要保留的列，value_vars表示需要stack的一组列

In [128]:
pivoted = df.pivot(index='ID',columns='Gender',values='Math')
pivoted.head()

Gender,F,M
ID,,
1101,NaN,34.0
1102,32.5,NaN
1103,NaN,87.2
1104,80.4,NaN
1105,84.8,NaN


In [129]:
result = pivoted.reset_index().melt(id_vars=['ID'],value_vars=['F','M'],value_name='Math')\
                     .dropna().set_index('ID').sort_index()
result.head()

,Gender,Math
ID,,
1101,M,34.0
1102,F,32.5
1103,M,87.2
1104,F,80.4
1105,F,84.8


In [130]:
df_m.set_index('ID').head()

,Gender,Math
ID,,
1101,M,34.0
1102,F,32.5
1103,M,87.2
1104,F,80.4
1105,F,84.8


In [131]:
#检验是否与展开前的df相同，可以分别将这些链式方法的中间步骤展开，看看是什么结果
result.equals(df_m.set_index('ID'))

True

### 2. 压缩与展开
#### （1）stack：这是最基础的变形函数，总共只有两个参数：level和dropna

In [132]:
df_s = pd.pivot_table(df,index=['Class','ID'],columns='Gender',values=['Height','Weight'])
df_s.head()

Height        Weight      
Gender          F      M      F     M
Class ID                             
C_1   1101    NaN  173.0    NaN  63.0
      1102  192.0    NaN   73.0   NaN
      1103    NaN  186.0    NaN  82.0
      1104  167.0    NaN   81.0   NaN
      1105  159.0    NaN   64.0   NaN

In [133]:
df_s.groupby('Class').head()

Height        Weight      
Gender          F      M      F     M
Class ID                             
C_1   1101    NaN  173.0    NaN  63.0
      1102  192.0    NaN   73.0   NaN
      1103    NaN  186.0    NaN  82.0
      1104  167.0    NaN   81.0   NaN
      1105  159.0    NaN   64.0   NaN
C_2   1201    NaN  188.0    NaN  68.0
      1202  176.0    NaN   94.0   NaN
      1203    NaN  160.0    NaN  53.0
      1204  162.0    NaN   63.0   NaN
      1205  167.0    NaN   63.0   NaN
C_3   1301    NaN  161.0    NaN  68.0
      1302  175.0    NaN   57.0   NaN
      1303    NaN  188.0    NaN  82.0
      1304    NaN  195.0    NaN  70.0
      1305  187.0    NaN   69.0   NaN
C_4   2401  192.0    NaN   62.0   NaN
      2402    NaN  166.0    NaN  82.0
      2403  158.0    NaN   60.0   NaN
      2404  160.0    NaN   84.0   NaN
      2405  193.0    NaN   54.0   NaN

In [134]:
# 使用stack函数,将列索引的次级索引展开为行索引的最内层索引
df_stacked = df_s.stack()
df_stacked.head()

Height  Weight
Class ID   Gender                
C_1   1101 M        173.0    63.0
      1102 F        192.0    73.0
      1103 M        186.0    82.0
      1104 F        167.0    81.0
      1105 F        159.0    64.0

In [136]:
df_stacked.groupby('Class').head()

Height  Weight
Class ID   Gender                
C_1   1101 M        173.0    63.0
      1102 F        192.0    73.0
      1103 M        186.0    82.0
      1104 F        167.0    81.0
      1105 F        159.0    64.0
C_2   1201 M        188.0    68.0
      1202 F        176.0    94.0
      1203 M        160.0    53.0
      1204 F        162.0    63.0
      1205 F        167.0    63.0
C_3   1301 M        161.0    68.0
      1302 F        175.0    57.0
      1303 M        188.0    82.0
      1304 M        195.0    70.0
      1305 F        187.0    69.0
C_4   2401 F        192.0    62.0
      2402 M        166.0    82.0
      2403 F        158.0    60.0
      2404 F        160.0    84.0
      2405 F        193.0    54.0

In [137]:
df_s.stack??
# Stack the prescribed level(s) from columns to index.
# 默认会把列的最内层索引变成行的最内层索引--其逆运算unstack则是把行的最内层索引变成列的最内层索引

Signature: df_s.stack(level=-1, dropna=True)
Source:   
    def stack(self, level=-1, dropna=True):
        """
        Stack the prescribed level(s) from columns to index.

        Return a reshaped DataFrame or Series having a multi-level
        index with one or more new inner-most levels compared to the current
        DataFrame. The new inner-most levels are created by pivoting the
        columns of the current dataframe:

          - if the columns have a single level, the output is a Series;
          - if the columns have multiple levels, the new index
            level(s) is (are) taken from the prescribed level(s) and
            the output is a DataFrame.

        The new index levels are sorted.

        Parameters
        ----------
        level : int, str, list, default -1
            Level(s) to stack from the column axis onto the index
            axis, defined as one index or label, or a list of indices
            or labels.
        dropna : bool, default True
    

#### stack函数可以看做将横向的索引放到纵向，因此功能类似与melt，参数level可指定变化的列索引是哪一层（或哪几层，需要列表）

In [139]:
df_stacked = df_s.stack(0) # 传入的参数 0 表示将df_s的(序号为0的)第一层索引放置到列的最内层
df_stacked.groupby('Class').head(3)

Gender                 F      M
Class ID                       
C_1   1101 Height    NaN  173.0
           Weight    NaN   63.0
      1102 Height  192.0    NaN
C_2   1201 Height    NaN  188.0
           Weight    NaN   68.0
      1202 Height  176.0    NaN
C_3   1301 Height    NaN  161.0
           Weight    NaN   68.0
      1302 Height  175.0    NaN
C_4   2401 Height  192.0    NaN
           Weight   62.0    NaN
      2402 Height    NaN  166.0

In [141]:
df_stacked.groupby('Class').count()

Gender,F,M
Class,,
C_1,10,10
C_2,10,10
C_3,10,10
C_4,8,2


#### (2) unstack：stack的逆函数，功能上类似于pivot_table

In [142]:
df_stacked.head()

Gender                 F      M
Class ID                       
C_1   1101 Height    NaN  173.0
           Weight    NaN   63.0
      1102 Height  192.0    NaN
           Weight   73.0    NaN
      1103 Height    NaN  186.0

In [149]:
df_stacked.index[:3] # 行索引有三层, 最内层索引是将df_s的列索引的第一层Height和Weight放置进来得到的

MultiIndex([('C_1', 1101, 'Height'),
            ('C_1', 1101, 'Weight'),
            ('C_1', 1102, 'Height')],
           names=['Class', 'ID', None])

In [145]:
result = df_stacked.unstack()#将行索引的最内层索引转置为列索引的最内层
result.head()

Gender          F             M       
           Height Weight Height Weight
Class ID                              
C_1   1101    NaN    NaN  173.0   63.0
      1102  192.0   73.0    NaN    NaN
      1103    NaN    NaN  186.0   82.0
      1104  167.0   81.0    NaN    NaN
      1105  159.0   64.0    NaN    NaN

In [146]:
result = df_stacked.unstack().swaplevel(1,0,axis=1).sort_index(axis=1) # 将列索引的层级交换顺序,就复原为df_s了
result.head()

Height        Weight      
Gender          F      M      F     M
Class ID                             
C_1   1101    NaN  173.0    NaN  63.0
      1102  192.0    NaN   73.0   NaN
      1103    NaN  186.0    NaN  82.0
      1104  167.0    NaN   81.0   NaN
      1105  159.0    NaN   64.0   NaN

In [144]:
result.equals(df_s)
#同样在unstack中可以指定level参数

True

## 三、哑变量与因子化

### 1. Dummy Variable（哑变量）
#### 这里主要介绍get_dummies函数，其功能主要是进行one-hot编码：

In [150]:
df_d = df[['Class','Gender','Weight']]
df_d.head()

,Class,Gender,Weight
0,C_1,M,63
1,C_1,F,73
2,C_1,M,82
3,C_1,F,81
4,C_1,F,64


#### 现在希望将上面的表格前两列转化为哑变量，并加入第三列Weight数值：

In [152]:
# 将分类变量转换为哑变量, 本质上就是把有多个唯一值的列, 在列的方向上按照唯一值展开, 然后在这些列中, 用0或者1来标识每一行是否取相应的值.
pd.get_dummies(df_d[['Class','Gender']]).head()
# 以第一行为例, 在原始df_d中,该行的Class列的值是C_1
# 在使用了get_dummies方法后, 将Class按照不同取值分成四列, 第一行在这四列的取值, 只有C_1这一列的值为1, 其他三列均为0.
# Gender的两列的情况类似
# prefix_sep="_"默认参数会自动将原始列名和列值用_连接起来做为新的列名--也可自行指定,详见参数说明.

,Class_C_1,Class_C_2,Class_C_3,Class_C_4,Gender_F,Gender_M
0,1,0,0,0,0,1
1,1,0,0,0,1,0
2,1,0,0,0,0,1
3,1,0,0,0,1,0
4,1,0,0,0,1,0


In [153]:
# 使用join方法加入第三列--join方法是后边第五章的内容
pd.get_dummies(df_d[['Class','Gender']]).join(df_d['Weight']).head()
#可选prefix参数添加前缀，prefix_sep添加分隔符

,Class_C_1,Class_C_2,Class_C_3,Class_C_4,Gender_F,Gender_M,Weight
0,1,0,0,0,0,1,63
1,1,0,0,0,1,0,73
2,1,0,0,0,0,1,82
3,1,0,0,0,1,0,81
4,1,0,0,0,1,0,64


In [154]:
pd.get_dummies??

Signature:
pd.get_dummies(
    data,
    prefix=None,
    prefix_sep='_',
    dummy_na=False,
    columns=None,
    sparse=False,
    drop_first=False,
    dtype=None,
) -> 'DataFrame'
Source:   
def get_dummies(
    data,
    prefix=None,
    prefix_sep="_",
    dummy_na=False,
    columns=None,
    sparse=False,
    drop_first=False,
    dtype=None,
) -> "DataFrame":
    """
    Convert categorical variable into dummy/indicator variables.

    Parameters
    ----------
    data : array-like, Series, or DataFrame
        Data of which to get dummy indicators.
    prefix : str, list of str, or dict of str, default None
        String to append DataFrame column names.
        Pass a list with length equal to the number of columns
        when calling get_dummies on a DataFrame. Alternatively, `prefix`
        can be a dictionary mapping column names to prefixes.
    prefix_sep : str, default '_'
        If appending prefix, separator/delimiter to use. Or pass a
        list or dictionar

### 2. factorize方法
#### 该方法主要用于自然数编码，并且缺失值会被记做-1，其中sort参数表示是否排序后赋值

In [22]:
codes, uniques = pd.factorize(['b', None, 'a', 'c', 'b'], sort=True)
display(codes)
display(uniques)

array([ 1, -1,  0,  2,  1])

array(['a', 'b', 'c'], dtype=object)

## 四、问题与练习

### 1. 问题
#### 【问题一】 上面提到了许多变形函数，如melt/crosstab/pivot/pivot_table/stack/unstack函数，请总结它们各自的使用特点。

#### 【问题二】 变形函数和多级索引是什么关系？哪些变形函数会使得索引维数变化？具体如何变化？
#### 【问题三】 请举出一个除了上文提过的关于哑变量方法的例子。
#### 【问题四】 使用完stack后立即使用unstack一定能保证变化结果与原始表完全一致吗？
#### 【问题五】 透视表中涉及了三个函数，请分别使用它们完成相同的目标（任务自定）并比较哪个速度最快。
#### 【问题六】 既然melt起到了stack的功能，为什么再设计stack函数？

### 2. 练习
#### 【练习一】 继续使用上一章的药物数据集：

In [166]:
d=pd.read_csv('data/Drugs.csv')
d.head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


In [190]:
len(d)

24062

#### (a) 现在请你将数据表转化成如下形态，每行需要显示每种药物在每个地区的10年至17年的变化情况，且前三列需要排序：
![avatar](picture/drug_pic.png)
#### (b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [191]:
# 失败的尝试 d[['YYYY','DrugReports']].pivot(columns='YYYY',values='DrugReports').fillna(0)

YYYY,2010,2011,2012,2013,2014,2015,2016,2017
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
24057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
24059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
24060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [175]:
# 失败的尝试 d[['State','COUNTY','SubstanceName']].drop_duplicates()

,State,COUNTY,SubstanceName
0,VA,ACCOMACK,Propoxyphene
1,OH,ADAMS,Morphine
2,PA,ADAMS,Methadone
3,VA,ALEXANDRIA CITY,Heroin
4,PA,ALLEGHENY,Hydromorphone
...,...,...,...
24034,PA,WASHINGTON,Furanyl fentanyl
24036,OH,WAYNE,Acryl fentanyl
24039,VA,WAYNESBORO CITY,Fentanyl
24041,PA,WESTMORELAND,Acryl fentanyl


In [198]:
# 失败的尝试 
d[['State','COUNTY','SubstanceName']].join(d[['YYYY','DrugReports']].pivot(columns='YYYY',values='DrugReports').fillna(0))#.sort_values(by=['State','COUNTY','SubstanceName'])
#.groupby('State','COUNTY','SubstanceName').sum()

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,VA,ACCOMACK,Propoxyphene,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,OH,ADAMS,Morphine,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,PA,ADAMS,Methadone,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,VA,ALEXANDRIA CITY,Heroin,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,PA,ALLEGHENY,Hydromorphone,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
24057,VA,WYTHE,Codeine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24058,VA,WYTHE,Hydrocodone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
24059,VA,WYTHE,Tramadol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
24060,PA,YORK,ANPP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [251]:
# 不需要使用聚合函数
pd.pivot_table(data=d,index=['State','COUNTY','SubstanceName'],columns='YYYY',values='DrugReports').sort_index()

YYYY                         2010  2011  2012  2013  2014  2015  2016  2017
State COUNTY  SubstanceName                                                
KY    ADAIR   Buprenorphine   NaN   3.0   5.0   4.0  27.0   5.0   7.0  10.0
              Codeine         NaN   NaN   1.0   NaN   NaN   NaN   NaN   1.0
              Fentanyl        NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN
              Heroin          NaN   NaN   1.0   2.0   NaN   1.0   NaN   2.0
              Hydrocodone     6.0   9.0  10.0  10.0   9.0   7.0  11.0   3.0
...                           ...   ...   ...   ...   ...   ...   ...   ...
WV    WOOD    Oxycodone       6.0   4.0  24.0   7.0   7.0  11.0   7.0   1.0
              Tramadol        NaN   NaN   NaN   NaN   1.0   NaN   4.0   3.0
      WYOMING Buprenorphine   NaN   1.0   1.0   1.0   NaN   NaN   NaN   1.0
              Hydrocodone     1.0   5.0   NaN   NaN   1.0   NaN   1.0   NaN
              Oxycodone       5.0   4.0  14.0  12.0   5.0   NaN   NaN   NaN

[6214 rows x 8 columns]

In [252]:
d_=pd.pivot_table(data=d,index=['State','COUNTY','SubstanceName'],columns='YYYY',values='DrugReports').sort_index().reset_index().fillna('-')#.drop()
d_

YYYY,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,-,3,5,4,27,5,7,10
1,KY,ADAIR,Codeine,-,-,1,-,-,-,-,1
2,KY,ADAIR,Fentanyl,-,-,1,-,-,-,-,-
3,KY,ADAIR,Heroin,-,-,1,2,-,1,-,2
4,KY,ADAIR,Hydrocodone,6,9,10,10,9,7,11,3
...,...,...,...,...,...,...,...,...,...,...,...
6209,WV,WOOD,Oxycodone,6,4,24,7,7,11,7,1
6210,WV,WOOD,Tramadol,-,-,-,-,1,-,4,3
6211,WV,WYOMING,Buprenorphine,-,1,1,1,-,-,-,1
6212,WV,WYOMING,Hydrocodone,1,5,-,-,1,-,1,-


In [253]:
# 怎么去掉index的name?
d_.set_index(keys=np.arange(len(d_)))

YYYY,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,-,3,5,4,27,5,7,10
1,KY,ADAIR,Codeine,-,-,1,-,-,-,-,1
2,KY,ADAIR,Fentanyl,-,-,1,-,-,-,-,-
3,KY,ADAIR,Heroin,-,-,1,2,-,1,-,2
4,KY,ADAIR,Hydrocodone,6,9,10,10,9,7,11,3
...,...,...,...,...,...,...,...,...,...,...,...
6209,WV,WOOD,Oxycodone,6,4,24,7,7,11,7,1
6210,WV,WOOD,Tramadol,-,-,-,-,1,-,4,3
6211,WV,WYOMING,Buprenorphine,-,1,1,1,-,-,-,1
6212,WV,WYOMING,Hydrocodone,1,5,-,-,1,-,1,-


In [257]:
d_.index.name,d_.columns.name,
# 不是index的name,是columns的

(None, 'YYYY')

In [260]:
# 重新设置columns 
d_.columns=[str(d).strip() for d in d_.columns]
d_.head()

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,-,3,5,4,27,5,7,10
1,KY,ADAIR,Codeine,-,-,1,-,-,-,-,1
2,KY,ADAIR,Fentanyl,-,-,1,-,-,-,-,-
3,KY,ADAIR,Heroin,-,-,1,2,-,1,-,2
4,KY,ADAIR,Hydrocodone,6,9,10,10,9,7,11,3


In [264]:
d_.columns,d_.columns.name
# 没有返回值了

(Index(['State', 'COUNTY', 'SubstanceName', '2010', '2011', '2012', '2013',
        '2014', '2015', '2016', '2017'],
       dtype='object'), None)

In [262]:
#(b)
d_.head()

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,-,3,5,4,27,5,7,10
1,KY,ADAIR,Codeine,-,-,1,-,-,-,-,1
2,KY,ADAIR,Fentanyl,-,-,1,-,-,-,-,-
3,KY,ADAIR,Heroin,-,-,1,2,-,1,-,2
4,KY,ADAIR,Hydrocodone,6,9,10,10,9,7,11,3


In [266]:
d_.columns[3:]

Index(['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017'], dtype='object')

In [275]:
d_.reset_index().melt(id_vars=['State','COUNTY','SubstanceName'],value_vars=d_.columns[3:],value_name='DrugReports').replace('-',np.nan).dropna().set_index(np.arange(len(d)))#.sort_index()

,State,COUNTY,SubstanceName,variable,DrugReports
0,KY,ADAIR,Hydrocodone,2010,6.0
1,KY,ADAIR,Methadone,2010,1.0
2,KY,ALLEN,Hydrocodone,2010,10.0
3,KY,ALLEN,Methadone,2010,4.0
4,KY,ALLEN,Oxycodone,2010,15.0
...,...,...,...,...,...
24057,WV,WOOD,Hydrocodone,2017,8.0
24058,WV,WOOD,Isobutyryl fentanyl,2017,3.0
24059,WV,WOOD,Oxycodone,2017,1.0
24060,WV,WOOD,Tramadol,2017,3.0


In [281]:
d_1=d_.reset_index().melt(id_vars=['State','COUNTY','SubstanceName'],value_vars=d_.columns[3:],value_name='DrugReports').replace('-',np.nan).dropna().set_index(np.arange(len(d)))\
.reindex(['variable','State','COUNTY','SubstanceName','DrugReports'],axis=1)
d_1.head()

,variable,State,COUNTY,SubstanceName,DrugReports
0,2010,KY,ADAIR,Hydrocodone,6.0
1,2010,KY,ADAIR,Methadone,1.0
2,2010,KY,ALLEN,Hydrocodone,10.0
3,2010,KY,ALLEN,Methadone,4.0
4,2010,KY,ALLEN,Oxycodone,15.0


In [282]:
d_1.columns=['YYYY','State','COUNTY','SubstanceName','DrugReports']
d_1.head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,KY,ADAIR,Hydrocodone,6.0
1,2010,KY,ADAIR,Methadone,1.0
2,2010,KY,ALLEN,Hydrocodone,10.0
3,2010,KY,ALLEN,Methadone,4.0
4,2010,KY,ALLEN,Oxycodone,15.0


In [290]:
d.sort_values(by=['YYYY','State','COUNTY','SubstanceName','DrugReports']).head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
401,2010,KY,ADAIR,Hydrocodone,6
101,2010,KY,ADAIR,Methadone,1
512,2010,KY,ALLEN,Hydrocodone,10
513,2010,KY,ALLEN,Methadone,4
5,2010,KY,ALLEN,Oxycodone,15


In [291]:
(d_1.sort_values(by=['YYYY','State','COUNTY','SubstanceName','DrugReports'])).equals(d.sort_values(by=['YYYY','State','COUNTY','SubstanceName','DrugReports']))
# 索引的原因

False

In [294]:
(d_1.sort_values(by=['YYYY','State','COUNTY','SubstanceName','DrugReports'])).values==(d.sort_values(by=['YYYY','State','COUNTY','SubstanceName','DrugReports'])).values
# 可能是年份列的数据类型不一致

array([[False,  True,  True,  True,  True],
       [False,  True,  True,  True,  True],
       [False,  True,  True,  True,  True],
       ...,
       [False,  True,  True,  True,  True],
       [False,  True,  True,  True,  True],
       [False,  True,  True,  True,  True]])

In [300]:
d_1.YYYY

0        2010
1        2010
2        2010
3        2010
4        2010
         ... 
24057    2017
24058    2017
24059    2017
24060    2017
24061    2017
Name: YYYY, Length: 24062, dtype: object

In [301]:
d.YYYY

0        2010
1        2010
2        2010
3        2010
4        2010
         ... 
24057    2017
24058    2017
24059    2017
24060    2017
24061    2017
Name: YYYY, Length: 24062, dtype: int64

In [303]:
(d_1.YYYY.astype('int64')==d.YYYY).sum()
# -- 不是24062...

21630

In [306]:
d_1.YYYY[d_1.YYYY.astype('int64')!=d.YYYY]

2377     2010
2378     2010
2379     2010
2380     2010
2381     2010
         ... 
20752    2017
20753    2017
20754    2017
20755    2017
20756    2017
Name: YYYY, Length: 2432, dtype: object

In [307]:
d.YYYY[d_1.YYYY.astype('int64')!=d.YYYY]
# --!

2377     2011
2378     2011
2379     2011
2380     2011
2381     2011
         ... 
20752    2016
20753    2016
20754    2016
20755    2016
20756    2016
Name: YYYY, Length: 2432, dtype: int64

#### 【练习二】 现有一份关于某地区地震情况的数据集，请解决如下问题：

In [308]:
e=pd.read_csv('data/Earthquake.csv')
e.head()

,日期,时间,维度,经度,方向,距离,深度,烈度
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


#### (a) 现在请你将数据表转化成如下形态，将方向列展开，并将距离、深度和烈度三个属性压缩：
![avatar](picture/earthquake_pic.png)
#### (b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [309]:
e.columns

Index(['日期', '时间', '维度', '经度', '方向', '距离', '深度', '烈度'], dtype='object')

In [311]:
# 先使用pivot_table将方向列展开为行,并设置显示的值为 深度 烈度 和距离
pd.pivot_table(e,index=['日期', '时间', '维度', '经度'],columns='方向', values=['距离', '深度', '烈度'])

深度                                    \
方向                                 east north north_east north_west south   
日期         时间          维度    经度                                             
1912.08.09 12:29:00 AM 40.60 27.20  NaN   NaN        NaN        NaN   NaN   
1912.08.10 12:23:00 AM 40.60 27.10  NaN   NaN        NaN        NaN   NaN   
           12:30:00 AM 40.60 27.10  NaN   NaN        NaN        NaN   NaN   
1912.08.11 12:19:04 AM 40.60 27.20  NaN   NaN        NaN        NaN   NaN   
           12:20:00 AM 40.60 27.10  NaN   NaN        NaN        NaN   NaN   
...                                 ...   ...        ...        ...   ...   
2017.07.26 12:50:05 AM 40.76 32.87  NaN   NaN        NaN        NaN   NaN   
2017.07.29 12:24:11 AM 37.12 28.59  NaN   5.0        NaN        NaN   NaN   
           12:33:23 AM 37.11 27.73  NaN   NaN        6.5        NaN   NaN   
2017.07.30 12:21:33 AM 37.39 31.37  NaN   NaN        NaN        NaN   NaN   
2017.07.31 12:11:31 AM 37.42 31.37  NaN   NaN        NaN        5.1   NaN   

                                                                烈度        ...  \
方向                                 south_east south_west west east north  ...   
日期         时间          维度    经度                                           ...   
1912.08.09 12:29:00 AM 40.60 27.20       16.0        NaN  NaN  NaN   NaN  ...   
1912.08.10 12:23:00 AM 40.60 27.10        NaN       15.0  NaN  NaN   NaN  ...   
           12:30:00 AM 40.60 27.10        NaN       15.0  NaN  NaN   NaN  ...   
1912.08.11 12:19:04 AM 40.60 27.20       30.0        NaN  NaN  NaN   NaN  ...   
           12:20:00 AM 40.60 27.10        NaN       15.0  NaN  NaN   NaN  ...   
...                                       ...        ...  ...  ...   ...  ...   
2017.07.26 12:50:05 AM 40.76 32.87        5.0        NaN  NaN  NaN   NaN  ...   
2017.07.29 12:24:11 AM 37.12 28.59        NaN        NaN  NaN  NaN   3.9  ...   
           12:33:23 AM 37.11 27.73        NaN        NaN  NaN  NaN   NaN  ...   
2017.07.30 12:21:33 AM 37.39 31.37        NaN        5.0  NaN  NaN   NaN  ...   
2017.07.31 12:11:31 AM 37.42 31.37        NaN        NaN  NaN  NaN   NaN  ...   

                                                     距离                   \
方向                                 south_west west east north north_east   
日期         时间          维度    经度                                            
1912.08.09 12:29:00 AM 40.60 27.20        NaN  NaN  NaN   NaN        NaN   
1912.08.10 12:23:00 AM 40.60 27.10        6.0  NaN  NaN   NaN        NaN   
           12:30:00 AM 40.60 27.10        5.2  NaN  NaN   NaN        NaN   
1912.08.11 12:19:04 AM 40.60 27.20        NaN  NaN  NaN   NaN        NaN   
           12:20:00 AM 40.60 27.10        4.5  NaN  NaN   NaN        NaN   
...                                       ...  ...  ...   ...        ...   
2017.07.26 12:50:05 AM 40.76 32.87        NaN  NaN  NaN   NaN        NaN   
2017.07.29 12:24:11 AM 37.12 28.59        NaN  NaN  NaN   0.5        NaN   
           12:33:23 AM 37.11 27.73        NaN  NaN  NaN   NaN        1.2   
2017.07.30 12:21:33 AM 37.39 31.37        3.5  NaN  NaN   NaN        NaN   
2017.07.31 12:11:31 AM 37.42 31.37        NaN  NaN  NaN   NaN        NaN   

                                                                                
方向                                 north_west south south_east south_west west  
日期         时间          维度    经度                                                 
1912.08.09 12:29:00 AM 40.60 27.20        NaN   NaN        4.3        NaN  NaN  
1912.08.10 12:23:00 AM 40.60 27.10        NaN   NaN        NaN        2.0  NaN  
           12:30:00 AM 40.60 27.10        NaN   NaN        NaN        2.0  NaN  
1912.08.11 12:19:04 AM 40.60 27.20        NaN   NaN        4.3        NaN  NaN  
           12:20:00 AM 40.60 27.10        NaN   NaN        NaN        2.0  NaN  
...                                       ...   ...        ...        ...  ...  
2017.07.26 12:50:05 AM 40.76 

In [314]:
# 将层次化的列索引 距离 深度 烈度 使用stack转置为列
e_=pd.pivot_table(e,index=['日期', '时间', '维度', '经度'],columns='方向', values=['距离', '深度', '烈度']).stack(level=0)
e_.head()

方向                                   east  north  north_east  north_west  \
日期         时间          维度   经度                                             
1912.08.09 12:29:00 AM 40.6 27.2 深度   NaN    NaN         NaN         NaN   
                                 烈度   NaN    NaN         NaN         NaN   
                                 距离   NaN    NaN         NaN         NaN   
1912.08.10 12:23:00 AM 40.6 27.1 深度   NaN    NaN         NaN         NaN   
                                 烈度   NaN    NaN         NaN         NaN   

方向                                   south  south_east  south_west  west  
日期         时间          维度   经度                                            
1912.08.09 12:29:00 AM 40.6 27.2 深度    NaN        16.0         NaN   NaN  
                                 烈度    NaN         6.7         NaN   NaN  
                                 距离    NaN         4.3         NaN   NaN  
1912.08.10 12:23:00 AM 40.6 27.1 深度    NaN         NaN        15.0   NaN  
                                 烈度    NaN         NaN         6.0   NaN

In [318]:
# 再查看和设置一下index columns的name 
e_.index.names

FrozenList(['日期', '时间', '维度', '经度', None])

In [319]:
e_.index.names=['日期', '时间', '维度', '经度', '地震参数']
e_.head()

方向                                     east  north  north_east  north_west  \
日期         时间          维度   经度   地震参数                                        
1912.08.09 12:29:00 AM 40.6 27.2 深度     NaN    NaN         NaN         NaN   
                                 烈度     NaN    NaN         NaN         NaN   
                                 距离     NaN    NaN         NaN         NaN   
1912.08.10 12:23:00 AM 40.6 27.1 深度     NaN    NaN         NaN         NaN   
                                 烈度     NaN    NaN         NaN         NaN   

方向                                     south  south_east  south_west  west  
日期         时间          维度   经度   地震参数                                       
1912.08.09 12:29:00 AM 40.6 27.2 深度      NaN        16.0         NaN   NaN  
                                 烈度      NaN         6.7         NaN   NaN  
                                 距离      NaN         4.3         NaN   NaN  
1912.08.10 12:23:00 AM 40.6 27.1 深度      NaN         NaN        15.0   NaN  
                                 烈度      NaN         NaN         6.0   NaN

In [320]:
# 填充缺失值
e_.fillna('-').head()

方向                                    east north north_east north_west south  \
日期         时间          维度   经度   地震参数                                          
1912.08.09 12:29:00 AM 40.6 27.2 深度      -     -          -          -     -   
                                 烈度      -     -          -          -     -   
                                 距离      -     -          -          -     -   
1912.08.10 12:23:00 AM 40.6 27.1 深度      -     -          -          -     -   
                                 烈度      -     -          -          -     -   

方向                                    south_east south_west west  
日期         时间          维度   经度   地震参数                             
1912.08.09 12:29:00 AM 40.6 27.2 深度           16          -    -  
                                 烈度          6.7          -    -  
                                 距离          4.3          -    -  
1912.08.10 12:23:00 AM 40.6 27.1 深度            -         15    -  
                                 烈度            -          6    -